In [1]:
from google.cloud import aiplatform
from datetime import datetime
import pkg_resources
from IPython.display import Markdown as md
from google.cloud import service_usage_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import artifactregistry_v1
from google.cloud import storage
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [3]:
REGION = 'us-central1'
EXPERIMENT = 'hello-app'

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)
su_client = service_usage_v1.ServiceUsageClient()
ar_client = artifactregistry_v1.ArtifactRegistryClient()
cb_client = cloudbuild_v1.CloudBuildClient()

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{EXPERIMENT}"
DIR = f"{EXPERIMENT}"

In [6]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'mg-ce-demos-main@mg-ce-demos.iam.gserviceaccount.com'

In [7]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/aiplatform.admin
roles/bigquery.admin
roles/editor
roles/storage.objectAdmin


In [8]:
artifactregistry = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
    )
).state.name


if artifactregistry == 'DISABLED':
    print(f'Artifact Registry is currently {artifactregistry} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Artifact Registry is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Artifact Registry already enabled for project: {PROJECT_ID}')

Artifact Registry already enabled for project: mg-ce-demos


In [9]:
cloudbuild = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
    )
).state.name


if cloudbuild == 'DISABLED':
    print(f'Cloud Build is currently {cloudbuild} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Build is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Build already enabled for project: {PROJECT_ID}')

Cloud Build already enabled for project: mg-ce-demos


### Local flow

In [10]:
SCRIPT_PATH = 'app.py'

with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python

import os

from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    target = os.environ.get('TARGET', 'World')
    return 'Hello {}!\n'.format(target)

if __name__ == "__main__":
    app.run(debug=True,host='0.0.0.0',port=int(os.environ.get('PORT', 8080)))
```

In [11]:
%%writefile 'requirements.txt'
Flask 
gunicorn

Overwriting requirements.txt


In [12]:
%%writefile 'Dockerfile'
FROM python:3

# Copy local code to the container image.
ENV APP_HOME /app
WORKDIR $APP_HOME
COPY . ./

RUN pip install -r requirements.txt

# Run the web service on container startup. Here we use the gunicorn
# webserver, with one worker process and 8 threads.
# For environments with multiple CPU cores, increase the number of workers
# to be equal to the cores available.
CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 app:app

Overwriting Dockerfile


In [13]:
%%writefile '.dockerignore'
Dockerfile
README.md
*.pyc
*.pyo
*.pyd
*.ipynb
__pycache__

Overwriting .dockerignore


In [14]:
!gcloud artifacts repositories create hello-repo \
    --project={PROJECT_ID} \
    --repository-format=docker \
    --location=us-central1 \
    --description="Docker repository"

Create request issued for: [hello-repo]
Waiting for operation [projects/mg-ce-demos/locations/us-central1/operations/ba
e66ecb-be52-45c8-831f-d7a0eab30549] to complete...done.                        
Created repository [hello-repo].


In [15]:
!gcloud builds submit \
  --tag us-central1-docker.pkg.dev/{PROJECT_ID}/hello-repo/helloworld-gke .

Creating temporary tarball archive of 7 file(s) totalling 44.1 KiB before compression.
Uploading tarball of [.] to [gs://mg-ce-demos_cloudbuild/source/1695047357.21794-ad06a953a2bc45c2a06e58da83d24788.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mg-ce-demos/locations/global/builds/9ac6b77c-c9e7-4995-af97-9efd375e9c4b].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/9ac6b77c-c9e7-4995-af97-9efd375e9c4b?project=633472233130 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "9ac6b77c-c9e7-4995-af97-9efd375e9c4b"

FETCHSOURCE
Fetching storage object: gs://mg-ce-demos_cloudbuild/source/1695047357.21794-ad06a953a2bc45c2a06e58da83d24788.tgz#1695047357766627
Copying gs://mg-ce-demos_cloudbuild/source/1695047357.21794-ad06a953a2bc45c2a06e58da83d24788.tgz#1695047357766627...
/ [1 files][  5.9 KiB/  5.9 KiB]                                                
Operation completed over 1 objects/5.9 KiB.
BUILD


In [16]:
!gcloud services enable container.googleapis.com

In [17]:
!gcloud container clusters create-auto helloworld-gke \
  --location us-central1

Note: The Pod address range limits the maximum size of the cluster. Please refer to https://cloud.google.com/kubernetes-engine/docs/how-to/flexible-pod-cidr to learn how to optimize IP address allocation.
Creating cluster helloworld-gke in us-central1... Cluster is being configured..
.⠧                                                                             
Creating cluster helloworld-gke in us-central1... Cluster is being health-check
ed (master is healthy)...done.                                                 
Created [https://container.googleapis.com/v1/projects/mg-ce-demos/zones/us-central1/clusters/helloworld-gke].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1/helloworld-gke?project=mg-ce-demos
kubeconfig entry generated for helloworld-gke.
NAME            LOCATION     MASTER_VERSION  MASTER_IP     MACHINE_TYPE  NODE_VERSION    NUM_NODES  STATUS
helloworld-gke  us-central1  1.27.3-gke.100  34.135.62.

In [18]:
!kubectl get nodes

NAME                                            STATUS   ROLES    AGE   VERSION
gk3-helloworld-gke-default-pool-aae8032f-td3b   Ready    <none>   49s   v1.27.3-gke.100
gk3-helloworld-gke-default-pool-ddf93a33-z1b6   Ready    <none>   49s   v1.27.3-gke.100


In [24]:
%%writefile 'deployment.yaml'


# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-app
spec:
  replicas: 1
  selector:
    matchLabels:
      app: hello
  template:
    metadata:
      labels:
        app: hello
    spec:
      containers:
      - name: hello-app
        # Replace $LOCATION with your Artifact Registry location (e.g., us-west1).
        # Replace $GCLOUD_PROJECT with your project ID.
        image: us-central1-docker.pkg.dev/mg-ce-demos/hello-repo/helloworld-gke:latest
        # This app listens on port 8080 for web traffic by default.
        ports:
        - containerPort: 8080
        env:
          - name: PORT
            value: "8080"
        resources:
          requests:
            memory: "1Gi"
            cpu: "500m"
            ephemeral-storage: "1Gi"
          limits:
            memory: "1Gi"
            cpu: "500m"
            ephemeral-storage: "1Gi"
---

Overwriting deployment.yaml


In [33]:
%%writefile 'service.yaml'

# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# The hello service provides a load-balancing proxy over the hello-app
# pods. By specifying the type as a 'LoadBalancer', Kubernetes Engine will
# create an external HTTP load balancer.

apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: LoadBalancer
  selector:
    app: hello
  ports:
  - port: 80
    targetPort: 8080
---

Overwriting service.yaml


In [25]:
!kubectl apply -f deployment.yaml
!kubectl apply -f service.yaml

deployment.apps/hello-app created


In [41]:
print('DEPLOYMENTS: \n')
!kubectl get deployments

print('\nPODS: \n')
!kubectl get pods

print('\nSERVICES: \n')
!kubectl get services

DEPLOYMENTS: 

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
hello-app   1/1     1            1           5m21s

PODS: 

NAME                         READY   STATUS    RESTARTS   AGE
hello-app-7ffbdd744f-ndfls   1/1     Running   0          5m19s

SERVICES: 

NAME         TYPE           CLUSTER-IP       EXTERNAL-IP     PORT(S)        AGE
hello        LoadBalancer   34.118.234.204   34.135.31.159   80:31310/TCP   3m6s
kubernetes   ClusterIP      34.118.224.1     <none>          443/TCP        36m


In [42]:
!curl 34.135.31.159

Hello World!


### Clean Up

copy and paste to terminal

gcloud container clusters delete helloworld-gke --location us-central1

copy and paste to terminal

gcloud artifacts docker images delete us-central1-docker.pkg.dev/mg-ce-demos/hello-repo/helloworld-gke

### GCS flow

In [ ]:
bucket = gcs.lookup_bucket(PROJECT_ID)
SOURCEPATH = f'{EXPERIMENT}'

In [ ]:
blob = bucket.blob(f'{SOURCEPATH}/app.py')
blob.upload_from_filename(SCRIPT_PATH)

In [ ]:
requirements = f"""
Flask 
gunicorn
"""

In [ ]:
blob = bucket.blob(f'{SOURCEPATH}/requirements.txt')
blob.upload_from_string(requirements)

In [ ]:
dockerfile = f"""
FROM python:3

# Copy local code to the container image.
ENV APP_HOME /app
WORKDIR $APP_HOME
COPY . ./

RUN pip install -r requirements.txt
EXPOSE 80
CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 app:app
"""

In [ ]:
blob = bucket.blob(f'{SOURCEPATH}/Dockerfile')
blob.upload_from_string(dockerfile)

In [ ]:
dockerignore = f"""
Dockerfile
README.md
*.pyc
*.pyo
*.pyd
__pycache__
"""

In [ ]:
blob = bucket.blob(f'{SOURCEPATH}/.dockerignore')
blob.upload_from_string(dockerignore)

In [ ]:
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

In [ ]:
docker_repo = None
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if f'{PROJECT_ID}' == repo.name.split('/')[-1]:
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar_client.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{PROJECT_ID}',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {EXPERIMENT} experiment that holds docker images.',
                name = f'{PROJECT_ID}',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES, 'experiment': EXPERIMENT}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

In [ ]:
docker_repo.name, docker_repo.format_.name

In [ ]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

In [ ]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{PROJECT_ID}/{SOURCEPATH}/*', '/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}', '/workspace']
    }    
)
# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}"]

In [ ]:
build

In [ ]:
operation = cb_client.create_build(
    project_id = PROJECT_ID,
    build = build
)

In [ ]:
response = operation.result()
response.status, response.artifacts

In [ ]:
!gcloud services enable container.googleapis.com

In [ ]:
!gcloud container clusters create-auto helloworld-gke location us-central1

In [ ]:
!kubectl get nodes

In [ ]:
#!gcloud compute instances list

In [ ]:
%%writefile 'stt_app/deployment.yaml'



In [ ]:
%%writefile 'stt_app/deployment.yaml'

apiVersion: extensions/v1beta1
    kind: Deployment
    metadata:
        name: stt-app
    spec:
        replicas: 1
        selector:
            matchLabels:
                app: stt-app
        template:
            metadata:
                labels:
                    app: stt-app
            spec:
                containers:
                - name: stt-app
                    image: us-central1-docker.pkg.dev/mg-ce-demos/mg-ce-demos/stt_app
                    ports:
                    - containerPort: 80
                    env:
                    - name: PORT
value: 80

In [ ]:
%%writefile 'stt_app/service.yaml'

apiVersion: v1
    kind: Service
    metadata:
        name: stt-app
    spec:
        type: LoadBalancer
        selector:
            app: stt-app
        ports:
        - port: 80
        targetPort: 80

In [ ]:
!kubectl apply -f deployment.yaml
!kubectl apply -f service.yaml